In [1]:
#! /usr/bin/env python

# to make size- mag plot for stars. 
#Want to use actual stars for modelling- not just reserve. 
#Also want to use one exposure and one ccd: https://cdcvs.fnal.gov/redmine/projects/deswlwg/wiki/PSF_Modeling
#Need flags for cadidate and psf star

from __future__ import print_function
import os
import numpy as np
#from read_psf_cats import read_data, band_combinations
import fitsio
import treecorr
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from astropy.io import fits


def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
   
#des scheme
colors = ['#601A4A', '#EE442F','#63ACBE']

c_2=rgb_to_hex((239, 96, 96))
c_3=rgb_to_hex((55, 94, 50)) 
c_23=rgb_to_hex((121, 183, 179)) 

In [2]:
def add_to_list(filter, vlist, value):
    if 'griz' in vlist.keys():
        print("1")
        vlist['griz'].append(value)
    if filter not in vlist.keys():
        print("2")
        vlist[filter] = []
    if 'riz' in vlist.keys() and 'g' not in filter:
        print("3")
        vlist['riz'].append(value)
    vlist[filter].append(value)
    print(vlist)

In [3]:
def get_data(exps, work, fwhm_list, med_size_list, mean_size_list, tag):

    import astropy.io.fits as pyfits
    import os
    #expinfo_file=s.path.join(cat_dir, exp + "exp_psf_cat_" + exp + ".fits")
  
    print(work)
    cat_dir = work
    print(exps)
   
    for exp in exps:
        i=0
        if i<10:
            cat_file = os.path.join(cat_dir, exp + "/exp_psf_cat_" + exp + ".fits")

            with fitsio.FITS(cat_file) as pyf:
                band=pyf['info']['band'].read()
                band=band[0]
                fwhm=np.median(pyf['info']['star_fwhm'].read())

                if pyf['stars']['star_flag'] != 0:
                    continue
                    
            print(band)
            add_to_list(band, fwhm_list, fwhm)
            i+=1

    print('\nFinished processing all exposures')

In [ ]:
file="test.dat" #exps.da
tag="exp_psf_cat_"
work="/global/project/projectdirs/des/wl/desdata/users/mjarvis/y3a1-v29/"
exps=np.loadtxt("/global/project/projectdirs/des/wl/desdata/users/mjarvis/y3a1-v29/exps.dat").astype(int)

print('work dir = ', work)

"""if args.file != '':
    print('Read file ',file)
    with open(file) as fin:
        data = [ line.split() for line in fin ]
    runs, exps = zip(*data)
else:
    runs = args.runs
    exps = args.exps
"""
fwhm_list = {}
med_size_list = {}
mean_size_list = {}

get_data(exps.astype(str), work, fwhm_list, med_size_list, mean_size_list, tag)
print(fwhm_list)


work dir =  /global/project/projectdirs/des/wl/desdata/users/mjarvis/y3a1-v29/
/global/project/projectdirs/des/wl/desdata/users/mjarvis/y3a1-v29/
['349809' '349810' '349811' ..., '492875' '492876' '492877']


In [ ]:
#plot_seeing(fwhm_list)
fwhm=fwhm_list
fig = plt.figure()
plt.minorticks_on()
ax = fig.add_subplot(111)
print('median seeing in r = ',np.median(fwhm['r']))
print('median seeing in i = ',np.median(fwhm['i']))
print('median seeing in z = ',np.median(fwhm['z']))

riz = np.concatenate([fwhm['r'], fwhm['i'], fwhm['z']])
print('median seeing in riz = ',np.median(riz))

nbins = 40
range = (0.7, 1.7)
width = (range[1]-range[0])/nbins
n, bins, p = ax.hist([fwhm['z'],fwhm['i'],fwhm['r']], bins=nbins, range=range, 
                     histtype='barstacked', fill=True,
                     color=[c_2,c_3,c_23], width=width)
ax.set_xlabel('Seeing FWHM (arcsec)')
ax.set_ylabel('Number of exposures')
ax.legend(reversed(p), ['r', 'i', 'z'], loc='upper right')
ax.set_xlim(*range)
plt.tight_layout()
plt.savefig('seeing.pdf')
plt.show()